<a href="https://colab.research.google.com/github/crypto-saac/sequence-to-sequence-architectures-of-character-level-machine-translation-/blob/main/Character_Level_recurrent_Seq_2_Seq_Model_BidirectionalGRU_(4).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

In [ ]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip
!!unzip fra-eng.zip

['Archive:  fra-eng.zip',
 '  inflating: _about.txt              ',
 '  inflating: fra.txt                 ']

In [ ]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = "fra.txt"

In [ ]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens (num_encoder_tokenes):", num_encoder_tokens)
print("Number of unique output tokens (num_decoder_tokens):", num_decoder_tokens)
print("Max sequence length for inputs (max_encoder_seq_length):", max_encoder_seq_length)
print("Max sequence length for outputs (max_decoder_seq_length):", max_decoder_seq_length)

input_token_index = dict([(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict([(char, i) for i, char in enumerate(target_characters)])

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1 :, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

Number of samples: 10000
Number of unique input tokens (num_encoder_tokenes): 70
Number of unique output tokens (num_decoder_tokens): 93
Max sequence length for inputs (max_encoder_seq_length): 14
Max sequence length for outputs (max_decoder_seq_length): 59


In [ ]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))

encoder = keras.layers.Bidirectional(keras.layers.GRU(latent_dim, return_state=True, return_sequences=True))  # Update return_sequences=True

# Process the input sequence
#encoder_outputs, forward_h, forward_c, backward_h, backward_c = encoder(encoder_inputs)
encoder_outputs, forward_h, backward_h= encoder(encoder_inputs)

# Concatenate the hidden states
state_h = keras.layers.Concatenate()([forward_h, backward_h]) 
#state_c = keras.layers.Concatenate()([forward_c, backward_c])

# Define the encoder states
encoder_states = [state_h]

# Define the decoder inputs
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# Define the decoder GRU
decoder_gru = keras.layers.GRU(latent_dim*2, return_sequences=True, return_state=True)

# Process the decoder inputs with the initial states from the encoder
decoder_outputs, _ = decoder_gru(decoder_inputs, initial_state=encoder_states)

# Define the decoder dense layer
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [ ]:
decoder_outputs

<KerasTensor: shape=(None, None, 93) dtype=float32 (created by layer 'dense')>

In [ ]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)


Epoch 1/100
125/125 [==============================] - 13s 27ms/step - loss: 1.1581 - accuracy: 0.7443 - val_loss: 0.8548 - val_accuracy: 0.7537
Epoch 2/100
125/125 [==============================] - 2s 17ms/step - loss: 0.6624 - accuracy: 0.8124 - val_loss: 0.6590 - val_accuracy: 0.8102
Epoch 3/100
125/125 [==============================] - 2s 19ms/step - loss: 0.5527 - accuracy: 0.8402 - val_loss: 0.5845 - val_accuracy: 0.8292
Epoch 4/100
125/125 [==============================] - 2s 18ms/step - loss: 0.5037 - accuracy: 0.8531 - val_loss: 0.5500 - val_accuracy: 0.8403
Epoch 5/100
125/125 [==============================] - 2s 17ms/step - loss: 0.4714 - accuracy: 0.8617 - val_loss: 0.5252 - val_accuracy: 0.8453
Epoch 6/100
125/125 [==============================] - 2s 17ms/step - loss: 0.4455 - accuracy: 0.8679 - val_loss: 0.5029 - val_accuracy: 0.8513
Epoch 7/100
125/125 [==============================] - 2s 17ms/step - loss: 0.4217 - accuracy: 0.8747 - val_loss: 0.4850 - val_accuracy

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from nltk.translate.bleu_score import corpus_bleu, SmoothingFunction

predictions = model.predict([encoder_input_data, decoder_input_data])

# Convert predictions to text
predicted_texts = []
for prediction in predictions:
    predicted_text = ""
    for timestep in prediction:
        # Get index of highest probability
        char_index = np.argmax(timestep)
        # Convert index to character
        char = target_characters[char_index]
        # Append character to predicted text
        predicted_text += char
        # Stop if end sequence character is generated
        if char == "\n":
            break
    predicted_texts.append(predicted_text)

# Convert target texts to list of lists
target_texts_list = [list(text) for text in target_texts]

# Convert predicted texts to list of lists
predicted_texts_list = [list(text) for text in predicted_texts]

# Calculate BLEU score
bleu_score = corpus_bleu([[text] for text in target_texts_list], predicted_texts_list)
print("GRU:")
print("BLEU score:", bleu_score)

313/313 [==============================] - 2s 6ms/step
GRU:
BLEU score: 0.7802928075728157
